In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import seaborn as sns

In [2]:
VOCAB_SIZE = 3  # hyperparameter depending on the task
HEAD_SIZE = 8
EMBED_DIM = 512
HEAD_DIM = EMBED_DIM // HEAD_SIZE
D_PROB = 0.1

In [3]:
input_emb = nn.Embedding(num_embeddings=VOCAB_SIZE, embedding_dim=EMBED_DIM)
output_emb = nn.Embedding(num_embeddings=VOCAB_SIZE, embedding_dim=EMBED_DIM)

In [4]:
input_emb

Embedding(3, 512)

In [5]:
test = torch.LongTensor([[0, 1, 2], [2, 1, 0]])
# input_emb(test)

In [6]:
# output_emb(test)

In [7]:
do = input_emb(test)

## Head

In [8]:
key = nn.Linear(in_features=EMBED_DIM, out_features=HEAD_DIM, bias=False)
query = nn.Linear(in_features=EMBED_DIM, out_features=HEAD_DIM, bias=False)
value= nn.Linear(in_features=EMBED_DIM, out_features=HEAD_DIM, bias=False)

# TODO: figure out register buffer
# TODO: prototype tril
register_buffer = ('mask', torch.tril(torch.ones(HEAD_DIM, HEAD_DIM), diagonal=1))

dropout = nn.Dropout(D_PROB)

# t for tokens, so I think this is words in this case
_, t, d_k = do.size() 
k = key(do)
q = query(do)
v = value(do)
qk = (q@k.transpose(-1, -2)) / d_k ** 0.5

mask = torch.tril(torch.ones(HEAD_DIM, HEAD_DIM))

# if mask:
qk = qk.masked_fill(mask[:t, :t] == 0, float('-inf'))

# 0 is among batches so ith, jth inputs in each batch add to 1, we don't want this
# 1 is through the columns, which is the word so maybe
# 2 is the word vector entirely
qk = F.softmax(qk, dim=-1)
attn = qk @ v

# dropout occurs at the end of the sublayer before adding residual connections and layernorming
attn
attn = dropout(attn)

In [9]:
from model import Head

In [10]:
head = Head(HEAD_DIM, EMBED_DIM, True)

In [11]:
head(do)

tensor([[[ 0.1171, -0.5505,  0.0581, -0.7633, -0.9508,  0.8183,  0.3414,
           0.1193, -1.4610, -0.0232,  0.6588,  0.4555,  0.3613,  1.0119,
          -0.3905, -0.0000,  0.4997, -0.1334, -0.5695, -0.0000,  0.3129,
           0.2666,  0.4904,  0.5295, -0.0000,  0.4692,  0.3543,  0.1892,
          -0.0409, -0.0519, -0.6427,  0.4843, -0.1171,  0.0576, -0.6077,
          -0.3501, -0.7259, -0.1448, -0.1822, -0.3695,  1.1058, -0.0706,
           0.1943, -0.0617, -0.5871, -0.4608,  0.0000, -0.7178,  0.5932,
           0.8951,  0.9512,  0.0851,  0.1158,  0.0858,  0.0000,  0.2757,
           0.5213,  0.1205,  0.2625,  0.2627,  0.0124, -0.3538, -0.0651,
           0.1656],
         [ 0.2210, -0.3557, -0.0000, -0.5374, -0.4832,  0.2423,  0.0770,
          -0.0420, -1.2833,  0.1929,  0.2659,  0.4930,  0.1157,  0.5163,
          -0.4020,  0.0780,  0.2587, -0.0155, -0.5388, -0.6044,  0.3078,
           0.2206,  0.4595,  0.3112, -0.0000,  0.0618,  0.2753,  0.1026,
          -0.2094, -0.3854, -0.

## MultiHeadAttention

## FeedForward